## 1. Import packages

In [1]:
#!pip install tweepy

In [2]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm

## 2. Twitter API authentication

In [3]:
consumer_api_key = os.environ["TWITTER_CONSUMER_API_KEY"]
consumer_api_secret = os.environ["TWITTER_CONSUMER_API_SECRET"]

In [4]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

In [5]:
api = tw.API(auth, wait_on_rate_limit=True)

## 3. Tweets query

### 3.1. Define the query

In [6]:
search_words = "#covid19 -filter:retweets"
date_since = "2020-03-01"
# Collect tweets
tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(7500)

### 3.2. Retreive the tweets

In [7]:
tweets_copy = []
for tweet in tqdm(tweets):
    tweets_copy.append(tweet)

7500it [15:38,  7.99it/s]


In [8]:
print(f"new tweets retrieved: {len(tweets_copy)}")

new tweets retrieved: 7500


## 4. Populate the dataset

In [9]:
tweets_df = pd.DataFrame()
for tweet in tqdm(tweets_copy):
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': tweet.text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                               'is_retweet': tweet.retweeted}, index=[0]))

100%|██████████| 7500/7500 [00:38<00:00, 193.52it/s]


In [10]:
tweets_df.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,Greylupine710,"New York, USA",Storyteller. Avid reader. Video gamer. Role-pl...,2017-08-12 09:19:21,2647,1969,94269,False,2020-08-02 21:51:28,"It took two days since the last time I posted,...",None,Twitter Web App,False
0,Jon Cohen,"Westchester County, NY","“I don’t mind being called a propagandist, so ...",2009-03-04 00:39:31,2600,4994,210678,False,2020-08-02 21:51:26,Coronavirus #updates: Kids make up 10% of new ...,"[updates, Louisiana]",Twitter Web App,False
0,UJ_roc,In Cognito near Toledo,,2012-06-01 13:02:30,110,359,130,False,2020-08-02 21:51:18,@marcorubio\n\nSort of like @realDonaldTrump's...,[Covid19],TweetCaster for Android,False
0,Dustin LaPres,Reality,Beauty is Truth and Truth is Beauty...Musician...,2012-04-19 15:16:48,814,3089,85981,False,2020-08-02 21:51:15,@TheJonIverson @joshgad Foreign Assistance? I ...,None,Twitter for Android,False
0,Brain Treenew,,We lived through Obama. You’ll live through T...,2017-12-14 03:15:14,568,644,18724,False,2020-08-02 21:51:13,#COVID19 #AmericaFirst I am tired of politici...,"[COVID19, AmericaFirst]",Twitter for iPhone,False


## 5. Save the data

### 5.1. Read past data

In [11]:
tweets_old_df = pd.read_csv("covid19_tweets.csv")
print(f"past tweets: {tweets_old_df.shape}")

past tweets: (59436, 13)


### 5.2. Merge past and present data

In [12]:
tweets_all_df = pd.concat([tweets_old_df, tweets_df], axis=0)
print(f"new tweets: {tweets_df.shape[0]} past tweets: {tweets_old_df.shape[0]} all tweets: {tweets_all_df.shape[0]}")

new tweets: 7500 past tweets: 59436 all tweets: 66936


### 5.3. Drop duplicates

In [13]:
tweets_all_df.drop_duplicates(subset = ["user_name", "date", "text"], inplace=True)
print(f"all tweets: {tweets_all_df.shape}")

all tweets: (66936, 13)


### 5.4. Export the updated data

In [14]:
tweets_all_df.to_csv("covid19_tweets.csv", index=False)